In [ ]:
#| default_exp base.tree


---
skip_showdoc: true
---

In [ ]:
#| hide
# import nbdev; nbdev.nbdev_export()

In [ ]:
#| exporti
from fastcore.utils import *
import numpy as np
import pandas as pd
from mltreelib.utils.functions import *
from mltreelib.data import Data

In [ ]:
#| hide

# todo: Add followings
# 1) Parallelization strategy
# 1) Monotone constraints
# 1) All the provided data constraints
# 1) How to enable randomization for Random forest and ExtraTree Regressors
# 1) Cost complexity tables
# 1) Implement Survival / Anova / Poission / Oblique / Functional /Model based partitions / Conditional inference trees 
# 1) Implement crossvlaidatoin as control

In [ ]:
#| hide
from fastcore.test import *

In [ ]:
#| exporti

class Node:
    """ A generics node for all tress"""

    def __init__(self,
                nodeid:int = None,     # As simple id of the node
                depth: int = None,       # depth of current node
                split: Union[str,int,list] = None,     # Split definitions, New implementation coming
                kids: Union[None, list] = None,  # New implementation coming
                surrogates: Union[None, list] = None,  # New implementation coming
                n_subnodes: int = 2,  # Default for CART behaviors, if None then would assume C4.5 / ID3 behavior
                info:str = None   # just a generic info about the node
                ) -> None:

        self.nodeid, self.split, self.kids, self.surrogates = nodeid, split, kids, surrogates
        self.n_subnodes, self.depth, self.addons = n_subnodes, depth, info  # added information needed for diagnosis
        self._estimator, self._loss, self._wt, self._grad, self._cp, self._error =  None, None, None, None, None, None

        self._validate()

    def _validate(self):
        ...

    def __str__(self):

        return f"Node(ID:{self.nodeid}, depth:{self.depth}, nchilds:{len(self.kids)}, surrogates:{len(self.surrogates)}, info:{self.info})"

    __repr__ = __str__


class DecisionNode():
    """Class that represents a decision node or leaf in the decision tree"""
    def __init__(self, 
                
                feature_i: Union[str,int]=None,  # Feature index which we want to use as the threshold measure.
                threshold: float =None,          # The value that we will compare feature values at feature_i against to determine the prediction.
                value=None,                      # The class prediction if classification tree, or float value if regression tree.
                true_branch=None,                # Next decision node for samples where features value met the threshold.
                false_branch=None,               # Next decision node for samples where features value did not meet the threshold.
                ):
        self.feature_i = feature_i    
        self.threshold= threshold             
        self.value = value                            #
        self.true_branch = true_branch  #
        self.false_branch = false_branch# 
        self.estimator =  None                        # An estimator that can run at leaf node 
        self.is_leaf = False                          # indicator weather current leaf is final leaf or not
        self.loss = None                              # loss of gradient boosting
        self.grad = None                              # gradient
        self.w = None                                 # weight
        self.resid = None                             # Residuals
     
        
    def estimate(self,x):
        return self.value if self.estimator is None else self.estimator.predict(x)

# Super class of RegressionTree and ClassificationTree
class BaseDecisionTree(object):
    """Super class of all kinds of tree."""
    def __init__(self, 
                objective:str = "class",   # defining objective of the whole tree building.
                min_samples_split:int = 20,   # The minimum number of samples needed to make a split when building a tree.
                min_sample_leaf: int = 10,    # Minimum sample required to have a leaf node
                max_compete: int = 4,         # the number of competitor splits retained in the output. It is useful to know not just which split was chosen, but which variable came in second, third, etc.
                min_impurity:float = 0.01,    # The minimum impurity required to split the tree further. this is equivalent to complexity in CART
                max_depth:int = float("inf"), # The maximum depth of a tree.
                max_surrogates: int = 2,         # more on this to make it generalized and not CART Specific
                surrogate_style: int = 2,        # more on this to make it generalized and not CART Specific
                parallelize: str = 'feature',    # follow form LightGBMs behavior
                tree_growth: str = 'cart',   # follow binary structure if cart else follow multiple structure as C4.5
                feature_weights: Union[float,int,list] = None, # weight of each feature in the split. Default is set to 1 for all features
                loss: Union[str, float] = None,      # String of loss or function which defines the loss. This amounts to Loss function that is used for Gradient Boosting models to calculate impurity.
                verbose:Union[bool,int]=False, # Verbosity for tree building
                digits:Union[int, None] = 2    # To round the values before doing a split 
                ):



        self.root = None  # Root node in dec. tree
        self.min_samples_split, self.min_sample_leaf = min_samples_split ,min_sample_leaf
        self.max_compete, self.min_impurity, self.max_depth  = max_compete, min_impurity, max_depth
        self.max_surrogates, self.surrogate_style = max_surrogates ,surrogate_style
        self.tree_growth, self.feature_weights = tree_growth, feature_weights

        # function defining how tree would grow
        self._node_function = None
        self._leaf_function = None
        self._split_function = None
        self._summary_function = None
        self._parallelization = parallelize
        

        # If y is one-hot encoded (multi-dim) or not (one-dim)
        self.one_dim = None

        # If Gradient Boost
        self._objective = objective
        self._loss = loss

        self.cat_features = None
        self.con_features = None
        self.features_ = None
        self.target_ = None

        # other things
        self.digits, self.verbose  = digits, verbose

        self._validate_params()

    def _validate_params(self):
        
        if self.loss not in ['mse','mae','rss', 'gini', 'entropy','friedman_mse','chisq','anova','var','std']:
            raise ValueError(f"{self.loss} is not a known loss")
        # todo : add other validations


    def fit(self, X, y, loss=None):
        """ Build decision tree """
        self.cat_features = X.select_dtypes(include=['object','category']).columns.tolist()
        self.con_features = [col for col in X.columns if col not in self.cat_features]
        self.features_ = X.columns.tolist()
        self.target_ = y.name
        self.one_dim = len(np.shape(y)) == 1
        # # Check if expansion of y is needed
        # if len(np.shape(y)) == 1:
        #     y = np.expand_dims(y, axis=1)

        # Add y as last column of X
        Xy = pd.concat((X, y), axis=1)
        self.root = self._build_tree(Xy)
        self.loss=None

    def _build_tree(self, Xy, current_depth=0,features=None):
        """ Recursive method which builds out the decision tree and splits X and respective y
        on the feature of X which (based on impurity) best separates the data"""

        largest_impurity = 0
        bst_estimator = None    
        bst_weights = None        
        best_criteria = {"feature_i": None, "threshold": None} # Feature index and threshold
        best_sets = {
            "leftXy": None,   # Xy of left subtree
            "rightXy": None # Xy of right subtree
            }# Subsets of the data
        
        n_samples, n_features = np.shape(Xy)
        if len(self.features_) + 1  == n_features:
            if self.verbose : print(f"\nWe are going good at {current_depth} depth")
        else:
            print('Something is wrong',n_samples, n_features, self.features_)

        if n_samples >= self.min_samples_split and current_depth <= self.max_depth:
            # Calculate the impurity for each feature
            for feature_i in self.features_:
                if self.verbose : print(f'Evaluating {feature_i}')
                # All values of feature_i       
                if feature_i in self.cat_features:
                    unique_values = get_sorted_cats(Xy[feature_i].values)
                    thresh = []
                else:
                    if self.digits is not None:
                        unique_values = np.unique(np.round(Xy[feature_i],self.digits))
                    else:
                        unique_values = np.unique(Xy[feature_i])
                    
                # Iterate through all unique values of feature column i and
                # calculate the impurity
                for threshold in unique_values:
                    # Divide X and y depending on if the feature value of X at index feature_i
                    # meets the threshold
                    if feature_i in self.cat_features:
                        thresh.append(threshold)
                        Xy1, Xy2 = divide_on_feature(Xy, feature_i, thresh)  
                    else:
                        Xy1, Xy2 = divide_on_feature(Xy, feature_i, threshold)

                    if len(Xy1) > 0 and len(Xy2) > 0:
                        
                        # Select the y-values of the two sets
                        # y1 = Xy1[:, n_features:]
                        # y2 = Xy2[:, n_features:]
                
                        # Calculate impurity
                        # impurity = self._impurity_calculation(y, y1, y2)
                        
                        # evaluate node function
                        impurity, estimator, weights, addons = self._node_function(Xy, Xy1, Xy2, target=self.target_,features=self.features_)

                        # If this threshold resulted in a higher information gain than previously
                        # recorded save the threshold value and the feature
                        # index
                        if impurity > largest_impurity:
                            largest_impurity, bst_estimator, bst_weights, bst_addons = impurity, estimator, weights, addons
                            best_criteria["feature_i"]= feature_i
                            best_criteria["threshold"] =  thresh if feature_i in self.cat_features else threshold
                            best_sets["leftXy"]=  Xy1 # X of left subtree
                            best_sets["rightXy"]=  Xy2 # X of left subtree
                            
                if self.verbose: print(f"Impurity Update at {best_criteria} with {largest_impurity}")
                            
                                
        if largest_impurity > self.min_impurity:
            if self.verbose: print(f'Best Impurity is with {best_criteria} with {largest_impurity} Split sizes: Left: {best_sets["leftXy"].shape} : Right {best_sets["rightXy"].shape}')
            # Build subtrees for the right and left branches
            true_branch = self._build_tree(best_sets["leftXy"], current_depth + 1)
            false_branch = self._build_tree(best_sets["rightXy"], current_depth + 1)
            node = DecisionNode(feature_i=best_criteria["feature_i"], threshold=best_criteria[
                                "threshold"], true_branch=true_branch, false_branch=false_branch,depth=current_depth)
            node.estimator = bst_estimator
            node.w = bst_weights
            node.addons = bst_addons
            return node
        else:
            if self.verbose: print(f"No Impurity update. Hence a leaf with {best_criteria} with {largest_impurity}")

        # We're at leaf => determine value
        leaf_value,estimator,weights,addons = self._leaf_function(Xy, target = self.target_, features=self.features_)
        leaf_node = DecisionNode(value=leaf_value)
        leaf_node.estimator = estimator
        leaf_node.w = weights
        leaf_node.addons = addons
        return leaf_node


    def predict_value(self, x, tree=None,verbose=False):
        """ Do a recursive search down the tree and make a prediction of the data sample by the
            value of the leaf that we end up at """

        if tree is None:
            tree = self.root

        # If we have a value (i.e we're at a leaf) => return value as the prediction
        if tree.value is not None:
            return tree.estimate(x)

        # Choose the feature that we will test
        if isinstance(x, pd.Series):
            feature_value = x.loc[tree.feature_i]
        else:
            feature_value = x[tree.feature_i].values[0]
        if verbose:
            print(f'Feature value for {tree.feature_i} :{feature_value}')

        # Determine if we will follow left or right branch
        branch = tree.false_branch
        if isinstance(feature_value, int) or isinstance(feature_value, float):
            if feature_value >= tree.threshold:  branch = tree.true_branch
        elif isinstance(tree.threshold,list):
            if feature_value in tree.threshold: branch = tree.true_branch
        elif feature_value == tree.threshold: 
            branch = tree.true_branch

        # Test subtree
        return self.predict_value(x, branch,verbose)

    def predict(self, X):
        """ Classify samples one by one and return the set of labels """
        y_pred = [self.predict_value(sample) for sample in X]
        return y_pred

    def print_tree(self, tree=None, indent=" "):
        """ Recursively print the decision tree """
        if not tree:
            tree = self.root

        # If we're at leaf => print the label
        if tree.value is not None:
            print (tree.value)
        # Go deeper down the tree
        else:
            # Print test
            print ("%s:%s? " % (tree.feature_i, tree.threshold))
            # Print the true scenario
            print ("%sT->" % (indent), end="")
            self.print_tree(tree.true_branch, indent + indent)
            # Print the false scenario
            print ("%sF->" % (indent), end="")
            self.print_tree(tree.false_branch, indent + indent)
            